# Advanced Retrieval with LangChain

In the following notebook, we'll explore various methods of advanced retrieval using LangChain!

We'll touch on:

- Naive Retrieval
- Best-Matching 25 (BM25)
- Multi-Query Retrieval
- Parent-Document Retrieval
- Contextual Compression (a.k.a. Rerank)
- Ensemble Retrieval
- Semantic chunking

We'll also discuss how these methods impact performance on our set of documents with a simple RAG chain.

There will be two breakout rooms:

- 🤝 Breakout Room Part #1
  - Task 1: Getting Dependencies!
  - Task 2: Data Collection and Preparation
  - Task 3: Setting Up QDrant!
  - Task 4-10: Retrieval Strategies
- 🤝 Breakout Room Part #2
  - Activity: Evaluate with Ragas

# 🤝 Breakout Room Part #1

## Task 1: Getting Dependencies!

We're going to need a few specific LangChain community packages, like OpenAI (for our [LLM](https://platform.openai.com/docs/models) and [Embedding Model](https://platform.openai.com/docs/guides/embeddings)) and Cohere (for our [Reranker](https://cohere.com/rerank)).

> You do not need to run the following cells if you are running this notebook locally. 

In [1]:
#!pip install -qU langchain langchain-openai langchain-cohere rank_bm25

We're also going to be leveraging [Qdrant's](https://qdrant.tech/documentation/frameworks/langchain/) (pronounced "Quadrant") VectorDB in "memory" mode (so we can leverage it locally in our colab environment).

In [2]:
#!pip install -qU qdrant-client

We'll also provide our OpenAI key, as well as our Cohere API key.

In [3]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

In [4]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

## Task 2: Data Collection and Preparation

We'll be using some reviews from the 4 movies in the John Wick franchise today to explore the different retrieval strategies.

These were obtained from IMDB, and are available in the [AIM Data Repository](https://github.com/AI-Maker-Space/DataRepository).

### Data Collection

We can simply `wget` these from GitHub.

You could use any review data you wanted in this step - just be careful to make sure your metadata is aligned with your choice.

In [5]:
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw1.csv -O john_wick_1.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw2.csv -O john_wick_2.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw3.csv -O john_wick_3.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw4.csv -O john_wick_4.csv

--2025-05-20 16:11:57--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8002::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19628 (19K) [text/plain]
Saving to: ‘john_wick_1.csv’

john_wick_1.csv     100%[===================>]  19.17K  --.-KB/s    in 0.001s  

2025-05-20 16:11:57 (30.1 MB/s) - ‘john_wick_1.csv’ saved [19628/19628]

--2025-05-20 16:11:57--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8000::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Len

### Data Preparation

We want to make sure all our documents have the relevant metadata for the various retrieval strategies we're going to be applying today.

- Self-Query: Wants as much metadata as we can provide
- Time-weighted: Wants temporal data

> NOTE: While we're creating a temporal relationship based on when these movies came out for illustrative purposes, it needs to be clear that the "time-weighting" in the Time-weighted Retriever is based on when the document was *accessed* last - not when it was created.

In [6]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from datetime import datetime, timedelta

documents = []

for i in range(1, 5):
  loader = CSVLoader(
      file_path=f"john_wick_{i}.csv",
      metadata_columns=["Review_Date", "Review_Title", "Review_Url", "Author", "Rating"]
  )

  movie_docs = loader.load()
  for doc in movie_docs:

    # Add the "Movie Title" (John Wick 1, 2, ...)
    doc.metadata["Movie_Title"] = f"John Wick {i}"

    # convert "Rating" to an `int`, if no rating is provided - assume 0 rating
    doc.metadata["Rating"] = int(doc.metadata["Rating"]) if doc.metadata["Rating"] else 0

    # newer movies have a more recent "last_accessed_at"
    doc.metadata["last_accessed_at"] = datetime.now() - timedelta(days=4-i)

  documents.extend(movie_docs)

Let's look at an example document to see if everything worked as expected!

In [7]:
documents[0]

Document(metadata={'source': 'john_wick_1.csv', 'row': 0, 'Review_Date': '6 May 2015', 'Review_Title': ' Kinetic, concise, and stylish; John Wick kicks ass.\n', 'Review_Url': '/review/rw3233896/?ref_=tt_urv', 'Author': 'lnvicta', 'Rating': 8, 'Movie_Title': 'John Wick 1', 'last_accessed_at': datetime.datetime(2025, 5, 17, 16, 11, 59, 155033)}, page_content=": 0\nReview: The best way I can describe John Wick is to picture Taken but instead of Liam Neeson it's Keanu Reeves and instead of his daughter it's his dog. That's essentially the plot of the movie. John Wick (Reeves) is out to seek revenge on the people who took something he loved from him. It's a beautifully simple premise for an action movie - when action movies get convoluted, they get bad i.e. A Good Day to Die Hard. John Wick gives the viewers what they want: Awesome action, stylish stunts, kinetic chaos, and a relatable hero to tie it all together. John Wick succeeds in its simplicity.")

## Task 3: Setting up QDrant!

Now that we have our documents, let's create a QDrant VectorStore with the collection name "JohnWick".

We'll leverage OpenAI's [`text-embedding-3-small`](https://openai.com/blog/new-embedding-models-and-api-updates) because it's a very powerful (and low-cost) embedding model.

> NOTE: We'll be creating additional vectorstores where necessary, but this pattern is still extremely useful.

In [8]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWick"
)

## Task 4: Naive RAG Chain

Since we're focusing on the "R" in RAG today - we'll create our Retriever first.

### R - Retrieval

This naive retriever will simply look at each review as a document, and use cosine-similarity to fetch the 10 most relevant documents.

> NOTE: We're choosing `10` as our `k` here to provide enough documents for our reranking process later

In [9]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

### A - Augmented

We're going to go with a standard prompt for our simple RAG chain today! Nothing fancy here, we want this to mostly be about the Retrieval process.

In [10]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

### G - Generation

We're going to leverage `gpt-4.1-nano` as our LLM today, as - again - we want this to largely be about the Retrieval process.

In [11]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model_name = "gpt-4o-mini")


### LCEL RAG Chain

We're going to use LCEL to construct our chain.

> NOTE: This chain will be exactly the same across the various examples with the exception of our Retriever!

In [12]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's see how this simple chain does on a few different prompts.

> NOTE: You might think that we've cherry picked prompts that showcase the individual skill of each of the retrieval strategies - you'd be correct!

In [13]:
naive_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Yes, people generally liked "John Wick." The reviews consistently highlight its engaging action sequences, strong performance by Keanu Reeves, and unique storytelling elements. For instance, several reviewers praised the film as one of the best action movies of the year or even the past decade, with comments about its stylish direction and intense pacing. While there are a few mixed opinions in the reviews, the overall sentiment is very positive, especially among fans of action films.'

In [14]:
naive_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10. Here is the URL to that review:\n\n- [A Masterpiece & Brilliant Sequel](https://www.imdb.com/review/rw4854296/?ref_=tt_urv)'

In [15]:
naive_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In "John Wick," an ex-hit-man named John Wick, played by Keanu Reeves, comes out of retirement to seek vengeance after gangsters kill his dog and steal his car. This occurs shortly after he has lost his wife, who had given him the dog as a final gift. As Wick pursues revenge, he finds himself targeted by numerous assassins due to his legendary status in the criminal underworld, leading to intense action and violent confrontations as he battles against those who wronged him. The film is noted for its stylized action sequences and the emotional motivation behind Wick\'s wrath.'

Overall, this is not bad! Let's see if we can make it better!

## Task 5: Best-Matching 25 (BM25) Retriever

Taking a step back in time - [BM25](https://www.nowpublishers.com/article/Details/INR-019) is based on [Bag-Of-Words](https://en.wikipedia.org/wiki/Bag-of-words_model) which is a sparse representation of text.

In essence, it's a way to compare how similar two pieces of text are based on the words they both contain.

This retriever is very straightforward to set-up! Let's see it happen down below!


In [16]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(documents)

We'll construct the same chain - only changing the retriever.

In [17]:
bm25_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at the responses!

In [18]:
bm25_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'People have had mixed opinions about the John Wick series. Some reviews are very positive, highlighting the action sequences and the film\'s artistic style, while others criticize the later installments for lacking plot and coherent storytelling. For example, "John Wick" received praise for its stylish action and engaging story, while "John Wick: Chapter 4" faced criticism for being excessively long with a weak plot. Overall, it seems that while many fans appreciate the series, there are also significant criticisms, especially of the later films.'

In [19]:
bm25_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'No, there are no reviews with a rating of 10 in the provided context.'

In [20]:
bm25_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In "John Wick," the story revolves around a retired hitman named John Wick, portrayed by Keanu Reeves. After the death of his wife, who had left him a puppy to help cope with her loss, John is devastated. When a group of criminals break into his home, steal his car, and kill his puppy, John is driven back into the violent world from which he retired. He seeks vengeance against those who wronged him, leading to a series of intense and beautifully choreographed action sequences. The film is noted for its emotional setup and the exploration of John\'s grief, coupled with its stylized action.'

It's not clear that this is better or worse - but the `I don't know` isn't great!

## Task 6: Contextual Compression (Using Reranking)

Contextual Compression is a fairly straightforward idea: We want to "compress" our retrieved context into just the most useful bits.

There are a few ways we can achieve this - but we're going to look at a specific example called reranking.

The basic idea here is this:

- We retrieve lots of documents that are very likely related to our query vector
- We "compress" those documents into a smaller set of *more* related documents using a reranking algorithm.

We'll be leveraging Cohere's Rerank model for our reranker today!

All we need to do is the following:

- Create a basic retriever
- Create a compressor (reranker, in this case)

That's it!

Let's see it in the code below!

In [21]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-english-v3.0")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)

Let's create our chain again, and see how this does!

In [22]:
contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [23]:
contextual_compression_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Yes, people generally liked "John Wick." Reviews highlight it as a fun, stylish, and well-executed action film with a strong performance by Keanu Reeves. It received positive comments for its action sequences, pacing, and unique take on the action genre, with some reviewers expressing that it\'s among the best action films they\'ve seen.'

In [24]:
contextual_compression_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10. Here is the URL to that review:\n\n- [Review by ymyuseda](https://www.imdb.com/review/rw4854296/?ref_=tt_urv)'

In [25]:
contextual_compression_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In the movie "John Wick," an ex-hit-man named John Wick, played by Keanu Reeves, seeks revenge after a group of gangsters kills his beloved dog and steals his car. The dog was a gift from his deceased wife, and its death triggers Wick\'s return to his violent past. As he embarks on a quest for vengeance, he faces numerous adversaries, leading to intense action sequences and shootouts. The film presents a story of loss and retribution, showcasing Wick as a legendary assassin who must confront an army of hitmen chasing him as a price is placed on his head. \n\nThe sequel, "John Wick 2," picks up after he resolves issues related to the Russian mafia. However, he is soon confronted by mobster Santino D\'Antonio, who holds a marker against Wick and demands his assistance in eliminating Santino\'s sister. After fulfilling this task, a contract is placed on Wick\'s life by Santino, prompting a new wave of violence and conflict as Wick seeks to settle the score.'

We'll need to rely on something like Ragas to help us get a better sense of how this is performing overall - but it "feels" better!

## Task 7: Multi-Query Retriever

Typically in RAG we have a single query - the one provided by the user.

What if we had....more than one query!

In essence, a Multi-Query Retriever works by:

1. Taking the original user query and creating `n` number of new user queries using an LLM.
2. Retrieving documents for each query.
3. Using all unique retrieved documents as context

So, how is it to set-up? Not bad! Let's see it down below!



In [26]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
)

In [27]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [28]:
multi_query_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Overall, yes, people generally liked the "John Wick" series. Reviews for the movies, particularly the first and fourth installments, highlight positive sentiments about the action sequences, Keanu Reeves\' performance, and the series\' ability to maintain a consistent quality. For instance, "John Wick 1" is praised for its innovative action and narrative simplicity, while "John Wick 4" received acclaim for its ambitious set pieces and overall entertainment value. However, there are some mixed reviews, especially regarding "John Wick 3," where some viewers felt that the series was losing momentum and becoming overly frenetic. Despite this, the majority of reviews lean towards a favorable reception of the franchise as a whole.'

In [29]:
multi_query_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there are reviews with a rating of 10. Here are the URLs to those reviews:\n\n1. [A Masterpiece & Brilliant Sequel](https://www.imdb.com/review/rw4854296/?ref_=tt_urv) (John Wick 3)\n2. [Entertaining Sequel](https://www.imdb.com/review/rw7582036/?ref_=tt_urv) (John Wick 2)'

In [30]:
multi_query_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In the movie "John Wick," the story follows John Wick, played by Keanu Reeves, who is a retired hitman. After the death of his wife, he attempts to find peace in his life. However, his life takes a dark turn when a group of thugs, led by the son of a Russian mobster he used to work for, breaks into his home. They beat him, steal his beloved car, and cruelly kill his puppy, which was a gift from his deceased wife.\n\nThis violent act triggers John Wick\'s quest for revenge, as he seeks to take down those who wronged him. The film is known for its kinetic action sequences, stylish stunts, and the portrayal of a relatable hero driven by a simple yet powerful premise: vengeance for the loss of what he loved most. Wick\'s journey plunges him back into the criminal underworld, where he faces numerous adversaries as he confronts the mobsters responsible for his suffering.'

## Task 8: Parent Document Retriever

A "small-to-big" strategy - the Parent Document Retriever works based on a simple strategy:

1. Each un-split "document" will be designated as a "parent document" (You could use larger chunks of document as well, but our data format allows us to consider the overall document as the parent chunk)
2. Store those "parent documents" in a memory store (not a VectorStore)
3. We will chunk each of those documents into smaller documents, and associate them with their respective parents, and store those in a VectorStore. We'll call those "child chunks".
4. When we query our Retriever, we will do a similarity search comparing our query vector to the "child chunks".
5. Instead of returning the "child chunks", we'll return their associated "parent chunks".

Okay, maybe that was a few steps - but the basic idea is this:

- Search for small documents
- Return big documents

The intuition is that we're likely to find the most relevant information by limiting the amount of semantic information that is encoded in each embedding vector - but we're likely to miss relevant surrounding context if we only use that information.

Let's start by creating our "parent documents" and defining a `RecursiveCharacterTextSplitter`.

In [31]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_docs = documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

We'll need to set up a new QDrant vectorstore - and we'll use another useful pattern to do so!

> NOTE: We are manually defining our embedding dimension, you'll need to change this if you're using a different embedding model.

In [32]:
client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="full_documents",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = Qdrant(
    collection_name="full_documents", embeddings=OpenAIEmbeddings(model="text-embedding-3-small"), client=client
)

/var/folders/9t/0z36p5hx6dd513bp6v3wxsz80000gn/T/ipykernel_99277/3574430551.py:8: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  parent_document_vectorstore = Qdrant(


Now we can create our `InMemoryStore` that will hold our "parent documents" - and build our retriever!

In [33]:
store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore = parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

By default, this is empty as we haven't added any documents - let's add some now!

In [34]:
parent_document_retriever.add_documents(parent_docs, ids=None)

We'll create the same chain we did before - but substitute our new `parent_document_retriever`.

In [35]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's give it a whirl!

In [36]:
parent_document_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'People have generally liked the John Wick series. While there are some negative reviews, such as one that described "John Wick 4" as "horrible" and criticized its plot and fight scenes, there are also many positive reviews praising the series for its action, choreography, and emotional depth. For example, one reviewer called "John Wick: Chapter 4" the best in the series and noted that it was a "wild ride." Overall, the series has a solid fan base and has been well received despite some criticisms.'

In [37]:
parent_document_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10. Here is the URL to that review:\n\n- [A Masterpiece & Brilliant Sequel](https://www.imdb.com/review/rw4854296/?ref_=tt_urv)'

In [38]:
parent_document_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In "John Wick," the story follows a retired hitman named John Wick, played by Keanu Reeves, who comes out of retirement to seek vengeance after gangsters kill his dog, a gift from his late wife, and steal his car. As the plot unfolds, Wick unleashes a wave of destruction on those responsible for his loss, showcasing his lethal skills and drawing the attention of various assassins and bounty hunters due to the price now on his head. The film is noted for its intense action, thrilling fight scenes, and the emotional depth behind Wick\'s motivations for revenge. \n\nIn the sequel, "John Wick: Chapter 2," which takes place shortly after the first film, Wick is forced back into the assassin world to pay off an old debt. He is tasked with helping an Italian crime lord and ends up killing numerous assassins while trying to navigate the complexities of the Assassin\'s Guild. The film opens with an intense car vs. bike chase and continues with high-octane action and violence as Wick confronts 

Overall, the performance *seems* largely the same. We can leverage a tool like [Ragas]() to more effectively answer the question about the performance.

## Task 9: Ensemble Retriever

In brief, an Ensemble Retriever simply takes 2, or more, retrievers and combines their retrieved documents based on a rank-fusion algorithm.

In this case - we're using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm.

Setting it up is as easy as providing a list of our desired retrievers - and the weights for each retriever.

In [39]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

We'll pack *all* of these retrievers together in an ensemble.

In [40]:
ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at our results!

In [41]:
ensemble_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Overall, people generally liked the "John Wick" series, especially the earlier films. Reviews highlight its stylish action, engaging performances (particularly by Keanu Reeves), and its ability to entertain within the action genre. Many reviewers consider the first film to be a standout, often citing its unique take on the action movie formula and its thrilling action sequences. \n\nHowever, there are also some mixed responses, particularly regarding the sequels, with some viewers expressing fatigue with the franchise or criticizing specific aspects of later entries. While many enjoyed the series, a few reviews noted that it felt repetitive or overly simplistic for their tastes. Overall, the consensus leans towards a positive reception, particularly for the first film.'

In [42]:
ensemble_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10. Here is the URL to that review:\n\n- [Review by ymyuseda](https://www.imdb.com/review/rw4854296/?ref_=tt_urv) titled "A Masterpiece & Brilliant Sequel" from the movie John Wick 3.'

In [43]:
ensemble_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In the movie "John Wick," an ex-hitman named John Wick seeks vengeance after a group of gangsters kills his dog and steals his car. The dog was a gift from his deceased wife, and its death serves as the catalyst for Wick to come out of retirement. As he embarks on his quest for revenge, it becomes clear that he is a legendary and feared assassin, and the gangsters are well aware of his capabilities. This leads to a series of violent encounters as other killers are drawn to him due to a bounty placed on his head. The film showcases intense action sequences and Wick\'s relentless pursuit of those who wronged him, resulting in a spiral of chaos and destruction as he fights against the Russian mob and other hitmen.'

## Task 10: Semantic Chunking

While this is not a retrieval method - it *is* an effective way of increasing retrieval performance on corpora that have clean semantic breaks in them.

Essentially, Semantic Chunking is implemented by:

1. Embedding all sentences in the corpus.
2. Combining or splitting sequences of sentences based on their semantic similarity based on a number of [possible thresholding methods](https://python.langchain.com/docs/how_to/semantic-chunker/):
  - `percentile`
  - `standard_deviation`
  - `interquartile`
  - `gradient`
3. Each sequence of related sentences is kept as a document!

Let's see how to implement this!

> NOTE: You do not need to run this cell if you're running this locally

In [44]:
#!pip install -qU langchain_experimental

We'll use the `percentile` thresholding method for this example which will:

Calculate all distances between sentences, and then break apart sequences of setences that exceed a given percentile among all distances.

In [52]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

Now we can split our documents.

In [53]:
semantic_documents = semantic_chunker.split_documents(documents)

Let's create a new vector store.

In [54]:
semantic_vectorstore = Qdrant.from_documents(
    semantic_documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWickSemantic"
)

We'll use naive retrieval for this example.

In [55]:
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

Finally we can create our classic chain!

In [56]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

And view the results!

In [57]:
semantic_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Yes, it appears that people generally liked the "John Wick" films. Many reviews praise the films for their action sequences, style, and Keanu Reeves\' performance. Reviewers describe "John Wick" as an entertaining and unique action film, highlighting its positive reception across the series. However, there are some mixed opinions, particularly regarding the later installments, with some criticism about changes in the franchise dynamics. Overall, the majority of feedback is favorable towards the films.'

In [58]:
semantic_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10. Here is the URL to that review:\n\n- [A Masterpiece & Brilliant Sequel](https://www.example.com/review/rw4854296/?ref_=tt_urv) (Review URL: /review/rw4854296/?ref_=tt_urv)'

In [59]:
semantic_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In "John Wick," the main character, played by Keanu Reeves, is a retired assassin seeking revenge after a group of criminals breaks into his home, beats him up, steals his beloved car, and kills his puppy. The dog was a gift from his late wife, making the loss even more poignant. As John Wick embarks on his vendetta, he faces off against various assassins and criminal elements from his past, unleashing a wave of destruction. The movie is characterized by its stylish action sequences, a strong emotional core, and a well-developed criminal underworld. Overall, it portrays Wick as a legendary figure whose lethal skills are sought after rather than praised in a world filled with killers.'

# 🤝 Breakout Room Part #2

#### 🏗️ Activity #1

Your task is to evaluate the various Retriever methods against eachother.

You are expected to:

1. Create a "golden dataset"
 - Use Synthetic Data Generation (powered by Ragas, or otherwise) to create this dataset
2. Evaluate each retriever with *retriever specific* Ragas metrics
 - Semantic Chunking is not considered a retriever method and will not be required for marks, but you may find it useful to do a "semantic chunking on" vs. "semantic chunking off" comparision between them
3. Compile these in a list and write a small paragraph about which is best for this particular data and why.

Your analysis should factor in:
  - Cost
  - Latency
  - Performance

> NOTE: This is **NOT** required to be completed in class. Please spend time in your breakout rooms creating a plan before moving on to writing code.

##### HINTS:

- LangSmith provides detailed information about latency and cost.

In [60]:
!mkdir data

mkdir: data: File exists


In [61]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31467    0 31467    0     0    99k      0 --:--:-- --:--:-- --:--:--   99k


In [62]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70634    0 70634    0     0   621k      0 --:--:-- --:--:-- --:--:--  627k


In [71]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


In [131]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

In [132]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

INFO:ragas.testset.synthesizers.multi_hop.abstract:found 1 clusters
INFO:ragas.testset.synthesizers.multi_hop.specific:found 41 clusters
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.op

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

In [149]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from qdrant_client import QdrantClient, models
from langchain.retrievers import EnsembleRetriever
from langchain_community.document_loaders import DirectoryLoader
from ragas.evaluation import evaluate
from ragas.testset import TestsetGenerator
import pandas as pd

# Data
path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_documents = text_splitter.split_documents(docs)

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Vector Store for retrieval
vectorstore = Qdrant.from_documents(
    split_documents,
    embeddings,
    location=":memory:",
    collection_name="ai_across_years"
)

naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

chat_model = ChatOpenAI(model_name = "gpt-4.1-nano")

# Define all retrieval chains
def build_chain(retriever):
    return (
        {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
        | RunnablePassthrough.assign(context=itemgetter("context"))
        | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
    )


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [152]:
# Define retrievers explicitly
bm25_retriever = BM25Retriever.from_documents(docs)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=CohereRerank(model="rerank-english-v3.0"), base_retriever=naive_retriever)
multi_query_retriever = MultiQueryRetriever.from_llm(retriever=naive_retriever, llm=chat_model)
parent_vectorstore = Qdrant.from_documents(
    documents=docs,
    embedding=embeddings,
    location=":memory:",
    collection_name="full_documents"
)
parent_retriever = ParentDocumentRetriever(
    vectorstore=parent_vectorstore,
    docstore=InMemoryStore(),
    child_splitter=RecursiveCharacterTextSplitter(chunk_size=200)
)

retrieval_chains = {
    "Naive Retriever": build_chain(naive_retriever),
    "BM25 Retriever": build_chain(bm25_retriever),
    # "Contextual Compression Retriever": build_chain(compression_retriever),
    "Multi Query Retriever": build_chain(multi_query_retriever),
    "Parent Document Retriever": build_chain(parent_retriever)
}

# Ensemble setup (only compatible retrievers)
ensemble_retriever = EnsembleRetriever(
    retrievers=[
        bm25_retriever,
        naive_retriever,
        multi_query_retriever,
        parent_retriever
    ],
    weights=[0.25, 0.25, 0.25, 0.25]
)
retrieval_chains["Ensemble Retriever"] = build_chain(ensemble_retriever)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [138]:
# generator_llm = ChatOpenAI(model_name = "gpt-4.1-nano")

# generator_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
# dataset_raw = generator.generate_with_langchain_docs(docs, testset_size=10)

# # Synthetic Dataset Creation
# generator = TestsetGenerator(llm=chat_model, embedding_model=embeddings)
# dataset_raw = generator.generate_with_langchain_docs(docs, testset_size=10)

# Evaluation-compatible Dataset (using raw testset)
# dataset = dataset_raw.samples

# Synthetic Dataset Creation

# generator = TestsetGenerator(llm=chat_model, embedding_model=embeddings)
# dataset_raw = generator.generate_with_langchain_docs(split_documents, testset_size=10)

# eval_dataset = convert_to_evaluation_dataset(dataset_raw.samples)
# eval_dataset = dataset.samples.to_evaluation_dataset()
eval_dataset = dataset.to_evaluation_dataset()



In [125]:
# dataset1 = eval_dataset.samples
# dataset1

[SingleTurnSample(user_input='What Stability AI do for LLMs and how they compare to other companies, like, is they making models or what, and is it hard for them to do it or not, and how much it cost, like, is it only for big companies or can anyone do it now?', retrieved_contexts=None, reference_contexts=['We don’t yet know how to build GPT-4 Vibes Based Development LLMs are really smart, and also really, really dumb Gullibility is the biggest unsolved problem Code may be the best application The ethics of this space remain diabolically complex My blog in 2023 Here’s the sequel to this post: Things we learned about LLMs in 2024. Large Language Models In the past 24-36 months, our species has discovered that you can take a GIANT corpus of text, run it through a pile of GPUs, and use it to create a fascinating new kind of software. LLMs can do a lot of things. They can answer questions, summarize documents, translate from one language to another, extract information and even write surpr

In [116]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What Stability AI do for LLMs and how they com...,[We don’t yet know how to build GPT-4 Vibes Ba...,Stability AI is one of the organizations that ...,single_hop_specifc_query_synthesizer
1,As an AI Product Manager focused on leveraging...,[I’m surprised that no-one has beaten the now ...,The grammar rules of programming languages lik...,single_hop_specifc_query_synthesizer
2,What notable developments regarding LLMs occur...,[Simon Willison’s Weblog Subscribe Stuff we fi...,2023 was the breakthrough year for Large Langu...,single_hop_specifc_query_synthesizer
3,How was ChatGPT utilized to process analytics ...,[Microsoft over this issue. The 69 page PDF is...,"According to the context, a screenshot of Plau...",single_hop_specifc_query_synthesizer
4,why prompt engineering matter for ai if prompt...,[<1-hop>\n\nskeptical as to their utility base...,prompt engineering matter for ai because good ...,multi_hop_abstract_query_synthesizer
5,How has the effectiveness of synthetic trainin...,[<1-hop>\n\nThe rise of inference-scaling “rea...,Synthetic training data has become a substanti...,multi_hop_abstract_query_synthesizer
6,How has the use of synthetic data in language ...,[<1-hop>\n\nThe rise of inference-scaling “rea...,The use of synthetic data in language model tr...,multi_hop_abstract_query_synthesizer
7,How have advances in efficiency and reduced en...,[<1-hop>\n\nThe rise of inference-scaling “rea...,Advances in efficiency and reduced energy cons...,multi_hop_abstract_query_synthesizer
8,How did Meta contribute to the evolution of la...,[<1-hop>\n\nWe don’t yet know how to build GPT...,Meta played a pivotal role in the evolution of...,multi_hop_specific_query_synthesizer
9,"Wut r sum of the main issues wit ChatGPT, incl...",[<1-hop>\n\nMicrosoft over this issue. The 69 ...,ChatGPT faces issues like questions about the ...,multi_hop_specific_query_synthesizer


In [120]:
# Add this before your loop to inspect the first row
if eval_dataset:
    first_row = eval_dataset[0]
    print(f"Available attributes: {dir(first_row)}")

Available attributes: ['__abstractmethods__', '__annotations__', '__class__', '__class_getitem__', '__class_vars__', '__copy__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__fields__', '__fields_set__', '__firstlineno__', '__format__', '__ge__', '__get_pydantic_core_schema__', '__get_pydantic_json_schema__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__pretty__', '__private_attributes__', '__pydantic_complete__', '__pydantic_computed_fields__', '__pydantic_core_schema__', '__pydantic_custom_init__', '__pydantic_decorators__', '__pydantic_extra__', '__pydantic_fields__', '__pydantic_fields_set__', '__pydantic_generic_metadata__', '__pydantic_init_subclass__', '__pydantic_parent_namespace__', '__pydantic_post_init__', '__pydantic_private__', '__pydantic_root_model__', '__pydantic_serializer__', '__pydantic_setattr_handlers__',

In [153]:
# Evaluation Loop
results = []

for name, chain in retrieval_chains.items():
    print(f"Evaluating {name}...")

    for test_row in eval_dataset:
        response = chain.invoke({"question": test_row.user_input})
        test_row.retrieved_contexts = [doc.page_content for doc in response["context"]]
        test_row.response = response["response"].content if hasattr(response["response"], "content") else response["response"]

    metric_results = evaluate(eval_dataset)
    metric_scores = metric_results._scores_dict  # Access internal dictionary directly

    results.append({
        "Retriever": name,
        **metric_scores
    })

results_df = pd.DataFrame(results)
print(results_df)


Evaluating Naive Retriever...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https

Evaluating:   0%|          | 0/48 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request

Evaluating BM25 Retriever...


...

Evaluating Ensemble Retriever...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:langchain.retrievers.multi_query:Generated queries: ['What are Large Language Models, and how has their development evolved over the past two to three years?', 'Can you explain the concept of Large Language Models and detail the recent advancements made in their development within the last 24 to 36 months?', 'How have Large Language Models changed over the last 2 to 3 years, and what are the key developments in their development during this period?']
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request:

Evaluating:   0%|          | 0/48 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

                   Retriever  \
0            Naive Retriever   
1             BM25 Retriever   
2      Multi Query Retriever   
3  Parent Document Retriever   
4         Ensemble Retriever   

                                    answer_relevancy  \
0  [0.9413424650320928, 0.9928351595489006, 0.0, ...   
1  [0.924436052773217, 0.9928351595489006, 0.8510...   
2  [0.9339160449803289, 0.9888276162585262, 0.917...   
3  [0.9205901715282856, 0.0, 0.0, 0.0, 0.0, 0.863...   
4  [0.9050695400471932, 0.9877577327244399, 0.917...   

                                   context_precision  \
0  [0.9765432098656928, 0.16666666665, 0.39285714...   
1  [0.99999999995, 0.99999999995, 0.99999999995, ...   
2  [0.9622222222126, 0.14285714284285714, 0.36666...   
3  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
4  [0.9510602823023567, 0.19949494948829968, 0.40...   

                                        faithfulness  \
0  [1.0, 1.0, 0.8181818181818182, 1.0, 1.0, 1.0, ...   
1  [1.0, 0.0, 1.0, 1.

In [156]:
results_df.to_csv("retriever_comparison_results.csv", index=False)



In [159]:
import pandas as pd
import json

# Load the CSV
# results_df 

# Parse list values
for column in results_df.columns[1:]:
    results_df[column] = results_df[column].apply(
        lambda x: json.loads(
            x.replace("np.float64", "")
             .replace("array", "")
             .replace("(", "")
             .replace(")", "")
             .replace("'", "")
             .strip()
        ) if isinstance(x, str) else x
    )

# Compute mean scores
mean_scores_df = results_df.copy()
for column in mean_scores_df.columns[1:]:
    mean_scores_df[column] = mean_scores_df[column].apply(
        lambda x: sum(x)/len(x) if isinstance(x, list) else x
    )

# Display the result
print("\nMean Scores by Retriever:\n")
print(mean_scores_df.to_string(index=False))



Mean Scores by Retriever:

                Retriever  answer_relevancy  context_precision  faithfulness  context_recall
          Naive Retriever          0.853039           0.775786      0.926643        0.843750
           BM25 Retriever          0.930038           1.000000      0.906871        0.958333
    Multi Query Retriever          0.932053           0.770912      0.926168        0.898975
Parent Document Retriever          0.305690           0.000000      0.221861        0.250000
       Ensemble Retriever          0.929137           0.715776      0.863634        0.906250


The Multi Query Retriever emerges as the most effective retrieval strategy for this dataset due to its superior balance across all evaluation metrics. It achieved the highest average score in answer relevancy (0.932), indicating its strong ability to fetch contextually appropriate and semantically aligned information in response to diverse queries. It also tied for the top spot in faithfulness (0.926), meaning its retrieved answers consistently remained true to the source content. While its context precision (0.771) was slightly lower than BM25’s perfect score, it maintained a strong showing in context recall (0.899), indicating it retrieved a broader and more informative range of supporting content. This combination suggests that the Multi Query Retriever effectively diversifies the retrieval process, likely due to its ability to reformulate queries through LLM guidance, thereby surfacing richer and more nuanced context spans. For use cases requiring high-quality answer generation and factual integrity—such as research assistants, knowledge discovery platforms, or enterprise QA systems—this retriever strikes the optimal trade-off between comprehensiveness, specificity, and truthfulness.